In [27]:
test = [{'url': 'https://lucid-source.com', 'header': '## Title: Lucid | IT Solutions and Digital Marketing Company in Erbil, Iraq', 'content': 'Lucid is an IT solutions and digital marketing company in Erbil, Iraq. We specialize in providing innovative solutions to help businesses grow and succeed in the digital age.'}]




In [ ]:
test[0]['header'][:100]


In [1]:
import uuid
import requests
from urllib.parse import urlparse
from services.knowledge_base.web_scrape import map_url, scrape_url
from services.chat.chat import llm_response
from typing import List, Dict
import openai
from humanloop import Humanloop
from dotenv import load_dotenv
import os

load_dotenv()

client = Humanloop(
    api_key=os.getenv("HUMANLOOP_API_KEY"),
)

async def create_opening_line(scraped_content: str) -> str:
    # Assuming you have OpenAI API setup
    user_prompt = f"""
    # Instructions:
    Based on this website content, create an engaging opening line for a customer service agent, like  <example_opening_line>"Welcome to WeCreate👋! Looking to boost your brand's online presence? What's your biggest goal right now?</example_opening_line>
    IMPORTANT: You must only respond with the opening line, no other text.
    # Website content:
    {scraped_content[:1000]}"""
    
    system_prompt = "You are an expert at creating opening lines for customer service agents. You are given a website content and you are tasked with creating a friendly opening line for a customer service agent."
    
    response = await llm_response(system_prompt, user_prompt)
    return response

async def create_agent_instructions(scraped_content: str) -> str:
    user_prompt = f"""Based on this website content, create a short sentence that begins with the name of the Company, and a short description saying what it does, like "WeCreate, a branding & digital marketing agency based Singapore, Hong Kong, and the UK."
    {scraped_content[:1000]}"""
    
    system_prompt = "You are an expert at creating instructions for customer service agents. You are given a website content and you are tasked with creating instructions for a customer service agent."
    
    response = await llm_response(system_prompt, user_prompt)
    return response

async def create_agents_from_urls(urls: List[str]):
    global scraped_content
    for url in urls:
        print("url:",url)
        # Get domain name for agent name
        domain = urlparse(url).netloc
        agent_name = domain.replace("www.", "")
        print("agent_name:",agent_name)
        # Map and scrape the URL
        mapped_url = await map_url(url)
        print("\n\nmapped_url:",mapped_url)
        scraped_content = await scrape_url(mapped_url[:30])
        web_content = ""
        for i in scraped_content:
            web_content += i['content']
        web_content = web_content[:10000]

        # Generate opening line from content
        opening_line = await create_opening_line(web_content)
        print("\n\nopening line:",opening_line)

        company_details = await create_agent_instructions(web_content)

        print("\n\ncompany details:",company_details)
        ## abstracted away lead form content, as that's determined by the user and stored in the metadata

        humanloop_prompt = client.prompts.get(
            id="pr_BiPy5GsclDI8uAzmHR8S7",
        )
        agent_instructions = humanloop_prompt.template[0]['content'].replace("{{company_details}}", company_details)

        # Create data source dict
        data_source = [{
            "id": str(uuid.uuid4()),
            "title": url,
            "data_type": "web"
        }]
        
        features = {
                    "lead_gen": {
                        "enabled": True
                    },
                    "callTransfer": {
                        "enabled": False
                    },
                    "notifyOnInterest": {
                        "enabled": False
                    },
                    "appointmentBooking": {
                        "enabled": False
                    }
                }
        
        form_fields = {
                "fields": [
                    {
                    "type": "text",
                    "label": "Full name"
                    },
                    {
                    "type": "email",
                    "label": "Email address"
                    }
                ]
            }
        
        """ POST to create new agent """
        # Prepare payload
        payload = {
            "agentName": agent_name,
            "agentPurpose": "text-chatbot-agent",
            "dataSource": data_source,
            "openingLine": opening_line,
            "language": "en-GB", 
            "voice": "Alex K",
            "voiceProvider": "cartesia",
            "instructions": agent_instructions,
            "features": features,
            "form_fields": form_fields,
            "company_name": agent_name
        }

        # POST request
        response = requests.post(
            "http://localhost:8000/api/v1/livekit/new_agent",
            json=payload,
            headers={"x-user-id": "user_2mmXezcGmjZCf88gT2v2waCBsXv"}
        )
        
        if response.status_code != 200:
            print(f"Failed to create agent for {url}: {response.text}")
        else:
            print(f"Successfully created agent for {url}")


# Example usage:
urls = [
    "https://lucid-source.com/"
]

await create_agents_from_urls(urls)

url: https://lucid-source.com/
agent_name: lucid-source.com
hello, map_url


mapped_url: ['https://lucid-source.com', 'https://lucid-source.com/about', 'https://lucid-source.com/blog', 'https://lucid-source.com/portfolio', 'https://lucid-source.com/careers', 'https://backend.lucid-source.com', 'https://lucid-source.com/products/grenton', 'https://lucid-source.com/software-service', 'https://lucid-source.com/network-service', 'https://lucid-source.com/about-hitex', 'https://lucid-source.com/career/10', 'https://lucid-source.com/career/17', 'https://lucid-source.com/career/11', 'https://lucid-source.com/career/15', 'https://lucid-source.com/career/14', 'https://lucid-source.com/career/16', 'https://lucid-source.com/career/9', 'https://lucid-source.com/career/12', 'https://lucid-source.com/career/19', 'https://lucid-source.com/career/21', 'https://lucid-source.com/terms-of-use', 'https://lucid-source.com/privacy-and-policy', 'https://lucid-source.com/services/it-solutions', 'https://lucid

### Recall Meeting bots

In [ ]:
import requests

# API endpoint
url = "https://us-west-2.recall.ai/api/v1/bot/"

# Your authorization token
token = "209a894215526cd61d70ed2f77df6cb39f6b0e4d"

# Request headers
headers = {
    "Authorization": f"Token {token}",
    "Content-Type": "application/json"
}

# Request payload
payload = {
    "meeting_url": "https://meet.google.com/stn-gobs-xcr",
    "bot_name": "Bot"
}

# Send POST request
response = requests.post(url, headers=headers, json=payload)

# Check response
print(response.status_code)
print(response.json())

In [ ]:
bot_joining = response.json()

bot_joining

In [ ]:
import requests

url = "https://us-west-2.recall.ai/api/v1/bot/e09e55e2-a067-473d-80be-18b9725ba7b8/"

headers = {
    "accept": "application/json",
    "Authorization": "209a894215526cd61d70ed2f77df6cb39f6b0e4d"
}

response = requests.get(url, headers=headers)

print(response.text)

In [ ]:
response.json()